# Import modules

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
import catboost as cb
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pprint
from copy import deepcopy
from typing import *
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier
from functools import reduce
from sklearn.preprocessing import TargetEncoder
from ktools.preprocessing.categorical_denoiser_prepreprocesser import CategoricalDenoiserPreprocessor
from ktools.metrics.fast_matthew_correlation_coefficient import fast_matthews_corr_coeff
from ktools.preprocessing.categorical_string_label_error_imputator import CategoricalLabelErrorImputator
from ktools.preprocessing.categorical_features_embedder import SortMainCategories
from ktools.preprocessing.kaggle_dataset_manager import KaggleDatasetManager
from ktools.utils.data_science_pipeline_settings import DataSciencePipelineSettings
from sklearn.linear_model import LogisticRegression
from ktools.fitting.cross_validate_then_test_sklearn_model import CrossValidateTestSklearnModel
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import KFold

import xgboost as xgb
from ktools.modelling.models.lgbm_model import LGBMModel
from ktools.modelling.models.xgb_model import XGBoostModel
from ktools.modelling.models.catboost_model import CatBoostModel

/Users/yuwei-1/anaconda3/envs/ktools_2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Main

In [180]:
import os

path = "/Users/yuwei-1/Documents/projects/Kaggle-tools/data/used_car_prices/oof_test_predictions"
# dir_list = os.listdir(path)

dir_list = ['basic_cat_73101.72',
            'rf_lgbm_tuned_72674.08',
            'gbdt_lgbm_tuned_72551.05',
            # 'goss_lgbm_tuned_72580.14',
            'gbdt_xgb_tuned_72510.52',
            'basic_xgb_74740.93',
            # 'basic_lgbm_72952.96',
            # 'cat_tuned_72764.5'
            ]

oof_list = []
test_list = []
for i, dir in enumerate(dir_list):
    oof_path = os.path.join(path, dir)
    df = pd.read_csv(oof_path + "/oof_predictions.csv", index_col=0).rename(columns={"oof_predictions" : f"prediction_{i}"})
    testdf = pd.read_csv(oof_path + "/test_predictions.csv", index_col=0).rename(columns={"test_prediction" : f"prediction_{i}"})
    oof_list += [df]
    test_list += [testdf]

In [181]:
dir_list

['basic_cat_73101.72',
 'rf_lgbm_tuned_72674.08',
 'gbdt_lgbm_tuned_72551.05',
 'gbdt_xgb_tuned_72510.52',
 'basic_xgb_74740.93']

In [182]:
meta_model_dataset = pd.concat(oof_list, axis=1)
meta_model_test = pd.concat(test_list, axis=1)

In [183]:
train_df, test_df = pd.read_csv("/Users/yuwei-1/Documents/projects/Kaggle-tools/data/used_car_prices/basic_train.csv"), pd.read_csv("/Users/yuwei-1/Documents/projects/Kaggle-tools/data/used_car_prices/basic_test.csv")

In [184]:
# from sklearn.metrics import make_scorer, root_mean_squared_error
# from sklearn.model_selection import RepeatedKFold
# from sklearn.pipeline import make_pipeline
# from sklearn.svm import SVR


# X = pd.read_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/data/used_car_prices/basic_train.csv', index_col=0)
# y = np.log(X.pop('price'))
# X['mileage'] = X.pop('milage')//100

# kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=0)
# scoring=make_scorer(lambda y, yh: root_mean_squared_error(np.exp(y), np.exp(yh)))

# model = make_pipeline(
#     TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
#     StandardScaler().set_output(transform='pandas'),
#     SVR(C=1000, gamma=0.0075, epsilon=0.1, max_iter=5000)
# )

# scores = -cross_val_score(model, X, y, scoring=scoring, cv=kfold, n_jobs=1)
# print(F'CV RMSE score: {np.mean(scores):.5f} ± {np.std(scores):.5f}')

In [200]:
from sklearn.linear_model import LinearRegression, Lasso


model = Lasso(alpha=1e4, positive=True)

num_splits = 5
eval_metrics = {"r2" : r2_score, "rmse" : lambda y, yhat : root_mean_squared_error(y, yhat)}
skf = KFold(n_splits=num_splits, shuffle=True, random_state=42)

cvt = CrossValidateTestSklearnModel(model,
                              eval_metrics,
                              skf,
                              num_splits)

modellist, cv_scores, test_scores = cvt.evaluate(meta_model_dataset,
                                                 train_df['price'],
                                                 meta_model_dataset,
                                                 train_df['price'])

Final Model r2: 0.154777
5-fold cross validation r2:  0.1545022531077287
Final Model rmse: 72461.102369
5-fold cross validation rmse:  72472.89565387438


In [201]:
cvt.model.coef_

array([0.06181176, 0.13556445, 0.23634809, 0.56638801, 0.05831875])

In [199]:
sub = pd.read_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/data/used_car_prices/sample_submission.csv', index_col=0)

# for i, mdl in enumerate(modellist):
#     sub[f'price_{i}'] = mdl.predict(meta_model_test)

# sub['price'] = sub.drop(columns=['price']).mean(axis=1)

sub['price'] = cvt.model.predict(meta_model_test)
sub['price'].to_csv('submissions/used_cars/used_car_submission_v27.csv')

In [133]:
sub

,price,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,price_9
id,,,,,,,,,,,
188533,16609.084353,16419.197917,16608.205909,16710.652168,16663.079840,16714.322255,16526.453652,16609.348374,16643.730147,16574.072374,16621.780894
188534,76614.823365,76288.899316,76208.100212,76246.298526,76847.731892,77012.094606,77203.616040,76293.799346,76548.438336,76818.233142,76681.022237
188535,49832.521073,49231.505745,49611.472638,49701.471493,49991.690462,50276.053693,50306.615955,49511.650845,49897.441144,49985.830087,49811.478666
188536,28706.923083,28509.486010,28372.404035,28558.922445,28884.661036,28920.115602,28903.372022,28370.940156,28736.425801,29034.581896,28778.321824
188537,32750.804033,33010.918871,32916.681099,33035.162869,32726.451074,32515.422509,32459.678827,32974.409617,32822.596982,32531.451688,32515.266795
...,...,...,...,...,...,...,...,...,...,...,...
314218,28055.990604,27892.579438,27823.549204,28043.974465,28193.134893,28210.384959,28182.347660,27784.025140,28162.632289,28295.549530,27971.728464
314219,48824.517498,48010.463321,48351.314572,48551.823880,49129.327256,49569.323615,49334.535988,48679.040309,48922.725875,48992.360494,48704.259672
314220,24824.020671,25364.332187,25049.729640,25150.970786,24763.248885,24404.243715,24318.049462,25160.384617,24841.856391,24542.726928,24644.664100


In [45]:
# def update(df):
    
#     t = 100
    
#     cat_c = ['brand','model','fuel_type','engine','transmission','ext_col','int_col','accident','clean_title']
#     re_ = ['model','engine','transmission','ext_col','int_col']
    
#     for col in re_:
#         df.loc[df[col].value_counts(dropna=False)[df[col]].values < t, col] = "noise"
        
#     for col in cat_c:
#         df[col] = df[col].fillna('missing')
#         df[col] = df[col].astype('category')
        
#     return df

# train  = update(train)
# # test   = update(test)

In [2]:
class ConvertToLower:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings):
        settings = deepcopy(original_settings)
        for col_name in settings.categorical_col_names:
            settings.combined_df[col_name] = settings.combined_df[col_name].str.lower()
        return settings

In [3]:
class UsedCarSpecificConverter:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings):
        settings = deepcopy(original_settings)
        def find_pattern(pattern, text):
            match = re.search(pattern, text)
            if match:
                return match.group(1)
            else:
                return None
            
        def transmission(text):
            if 'a/t' in text or 'at' in text or 'automatic' in text:
                return 'automatic transmission'
            elif 'm/t' in text or 'mt' in text or 'manual' in text:
                return 'manual transmission'
            elif 'cvt' in text:
                return 'continuously variable transmission'
            else:
                return 'other'

        def camshafts(text):
            if 'dohc' in text:
                # double overhead camshaft
                return 'dohc'
            elif 'sohc' in text:
                #single overhead camshaft
                return 'sohc'
            elif 'ohv' in text:
                # overhead valve
                return 'ohv'
            elif 'vtec' in text:
                # variable valve timing and lift electronic control
                return 'vtec'
            else:
                return 'other'

        def injection(text):
            if 'ddi' in text:
                #direct diesel injection
                return 'ddi'
            elif 'gdi' in text:
                #gasoline direct injection
                return 'gdi'
            elif 'mpfi' in text:
                # multi-point fuel injection
                return 'mpfi'
            elif 'pdi' in text:
                # port fuel injection
                return 'pdi'
            elif 'tfsi' in text or 'tsi' in text:
                # turbo stratified injection
                return 'tfsi'
            elif 'gtdi' in text:
                # gasoline turbocharged direct injection
                return 'gtdi'
            elif 'sidi' in text:
                # spark ignition direct injection
                return 'sidi'
            else:
                return 'other'
            
        pattern = r'(\d*\.?\d+)\s*hp'
        settings.combined_df['horsepower'] = settings.combined_df['engine'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        pattern = r'(\d*\.?\d+)\s*(l|liter)'
        settings.combined_df['liters'] = settings.combined_df['engine'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        pattern = r'(\d*\.?\d+)\s*cylinder'
        settings.combined_df['cylinders'] = settings.combined_df['engine'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        pattern = r'(\d*\.?\d+)\s*(-speed|speed)'
        settings.combined_df['speed'] = settings.combined_df['transmission'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        
        settings.combined_df['injection'] = settings.combined_df['engine'].apply(lambda x : injection(x)).astype('object')
        settings.combined_df['camshaft'] = settings.combined_df['engine'].apply(lambda x : camshafts(x)).astype('object')
        settings.combined_df['transmission_clean'] = settings.combined_df['transmission'].apply(lambda x : transmission(x)).astype('object')
        settings.combined_df.loc[(settings.combined_df['model'].str.contains('model y|model x|model s|model 3', regex=True)), 'fuel_type'] = 'electric'
        settings.combined_df.loc[(settings.combined_df['model'].str.contains('electric')), 'fuel_type'] = 'electric'


        expensive_ext_color = ['blue caelum','dark sapphire','bianco monocerus','c / c',
                               'ice','tempest','beluga black','bianco icarus metallic','blu eleos',
                               'shadow black','nero noctis','sandstone metallic','lizard green','balloon white','onyx',
                               'donington grey metallic','china blue','diamond white','rosso corsa',
                                'granite','rosso mars metallic',
                                'carpathian grey','kemora gray metallic','grigio nimbus','dash','bianco isis','python green',
                                'fountain blue','custom color','vega blue','designo magno matte',
                                'brands hatch gray metallic',
                                'rift metallic','gentian blue metallic',
                                'arancio borealis','blue',
                                'aventurine green metallic',
                                'apex blue','daytona gray pearl effect',
                                'daytona gray pearl effect w/ black roof','matte white',
                                'carpathian grey premium metallic','blue metallic','santorini black metallic',
                                'quartzite grey metallic','carrara white metallic','black',
                                'kinetic blue',
                                'nero daytona']

        expensive_int_color = ['dark auburn',
                            'hotspur',
                            'cobalt blue',
                            'beluga hide',
                            'linen',
                            'beluga',
                            'black / brown',
                            'nero ade',
                            'sahara tan',
                            'portland']
        
        
        
        settings.combined_df['expensive_ext_col'] = settings.combined_df['ext_col'].isin(expensive_ext_color).astype(int)
        settings.combined_df['expensive_int_col'] = settings.combined_df['int_col'].isin(expensive_int_color).astype(int)
        settings.combined_df['twin_turbo'] = settings.combined_df['engine'].str.contains('twin turbo').astype(int)
        settings.combined_df['turbo'] = settings.combined_df['engine'].str.contains('turbo').astype(int)
        settings.combined_df['length_model'] = settings.combined_df['model'].apply(lambda x : len(x))
        settings.combined_df['length_ext_col'] = settings.combined_df['ext_col'].apply(lambda x : len(x))
        settings.combined_df['length_int_col'] = settings.combined_df['int_col'].apply(lambda x : len(x))
        
        clean_colors = ['ext_col', 'int_col']
        string_imputator = CategoricalLabelErrorImputator(verbose=True)
        settings.combined_df[['basic_ext_color', 'basic_int_color']] = string_imputator.impute(settings.combined_df[clean_colors],
                                                                                                                    clean_colors,
                                                                                                                    1500)
        
        settings.combined_df['basic_ext_color'] = settings.combined_df['basic_ext_color'].astype('object')
        settings.combined_df['basic_int_color'] = settings.combined_df['basic_int_color'].astype('object')
        # threshold = 2.5e6
        # print("num removed: ", (settings.combined_df['price'] > threshold).sum())
        # settings.combined_df = settings.combined_df[(settings.combined_df['price'] < threshold) | (settings.combined_df['price'].isna())]
        
        # settings.training_col_names += ['horsepower', 
        #                                 'injection',
        #                                 'camshaft', 
        #                                 'cylinders', 
        #                                 'expensive_ext_col', 
        #                                 'expensive_int_col', 
        #                                 'twin_turbo', 
        #                                 'turbo',
        #                                 'transmission_clean',
        #                                 'speed',
        #                                 'basic_ext_color',
        #                                 'basic_int_color',
        #                                 'liters',
        #                                 'length_model',
        #                                 'length_ext_col',
        #                                 'length_int_col'
        #                                 ]
        
        re_ = ['model','engine','transmission','ext_col','int_col']
        for col in re_:
            settings.combined_df.loc[settings.combined_df[col].value_counts(dropna=False)[settings.combined_df[col]].values < 100, col] = "noise"
        
        # settings.categorical_col_names += ['injection',
        #                                    'camshaft',
        #                                    'transmission_clean',
        #                                    'basic_ext_color',
        #                                    'basic_int_color']
        
        return settings

In [4]:
class BasicUsedCarSpecificConverter:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings):
        settings = deepcopy(original_settings)
        def find_pattern(pattern, text):
            match = re.search(pattern, text)
            if match:
                return match.group(1)
            else:
                return None

        pattern = r'(\d*\.?\d+)\s*hp'
        settings.combined_df['horsepower'] = settings.combined_df['engine'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        pattern = r'(\d*\.?\d+)\s*(l|liter)'
        settings.combined_df['liters'] = settings.combined_df['engine'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        pattern = r'(\d*\.?\d+)\s*cylinder'
        settings.combined_df['cylinders'] = settings.combined_df['engine'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        pattern = r'(\d*\.?\d+)\s*(-speed|speed)'
        settings.combined_df['speed'] = settings.combined_df['transmission'].apply(lambda x : find_pattern(pattern, x)).astype('float64')

        settings.training_col_names += ['horsepower',
                                        'liters',
                                        'cylinders',
                                        'speed'
                                        ]
        return settings

In [5]:
class BackPackerMinimalProcessing:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings):

        settings = deepcopy(original_settings)
        settings.update()
        def update(df):
            t = 100
            re_ = ['model','engine','transmission','ext_col','int_col']
            for col in re_:
                df.loc[df[col].value_counts(dropna=False)[df[col]].values < t, col] = "noise"
            return df
        
        settings.train_df = update(settings.train_df)
        settings.test_df = update(settings.test_df)
        settings.combined_df = pd.concat([settings.train_df, settings.test_df], keys=['train', 'test'])
        return settings

In [6]:
class SergeyConverter:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings):
        settings = deepcopy(original_settings)
        fuel_type_dict = {
            'Gasoline': 0,
            'Hybrid': 1,
            'E85 Flex Fuel': 2,
            'uknown': 3,
            'Diesel': 4,
            'dash': 5,
            'Plug-In Hybrid': 6,
            'not supported': 7
        }

        accident_dict = {
            'None reported': 0,
            'At least 1 accident or damage reported': 1,
            'uknown': 2
        }

        clean_title_dict = {
            'Yes': 0,
            'uknown': 1
        }

        expensive_ext_color = ['Blue Caelum', 'Dark Sapphire', 'Bianco Monocerus', 'C / C', 'Ice',
            'Tempest', 'Beluga Black', 'Bianco Icarus Metallic', 
            'BLU ELEOS', 'Shadow Black', 'Nero Noctis', 'Sandstone Metallic',
            'Lizard Green', 'Balloon White', 'Onyx', 'Donington Grey Metallic',
            'China Blue', 'Diamond White', 'Rosso Corsa', 'Granite',
            'Rosso Mars Metallic', 'Carpathian Grey', 'Kemora Gray Metallic',
            'Grigio Nimbus', 'dash', 'Bianco Isis', 'Python Green', 'Fountain Blue',
            'Custom Color', 'Vega Blue', 'Designo Magno Matte',
            'Brands Hatch Gray Metallic', 'Rift Metallic', 'Gentian Blue Metallic',
            'Arancio Borealis', 'BLUE', 'Aventurine Green Metallic', 'Apex Blue',
            'Daytona Gray Pearl Effect', 'Daytona Gray Pearl Effect w/ Black Roof',
            'Matte White', 'Carpathian Grey Premium Metallic', 'Blue Metallic',
            'Santorini Black Metallic', 'Quartzite Grey Metallic',
            'Carrara White Metallic', 'BLACK', 'Kinetic Blue', 'Nero Daytona']

        expensive_int_color = ['Dark Auburn', 'Hotspur', 'Cobalt Blue', 'Beluga Hide', 'Linen',
                            'Beluga', 'Black / Brown', 'Nero Ade', 'Sahara Tan', 'Portland']

        expensive_hp = [443.0, 473.0, 493.0, 502.0, 521.0, 542.0, 543.0, 571.0, 572.0, 573.0, 580.0,
                        591.0, 602.0, 611.0, 616.0, 620.0, 624.0, 640.0, 641.0, 651.0, 710.0, 715.0, 760.0, 788.0, 797.0]



        def encode_columns(df):
            df['fuel_type_encoded'] = df['fuel_type'].map(fuel_type_dict)
            df['accident_encoded'] = df['accident'].map(accident_dict)
            df['clean_title_encoded'] = df['clean_title'].map(clean_title_dict)
            df['expensive_color_ext_encoded'] = df.ext_col.isin(expensive_ext_color).astype(int)
            df['expensive_color_int_encoded'] = df.int_col.isin(expensive_int_color).astype(int)
            df['expensive_hp'] = df['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float).isin(expensive_hp).astype(int)
            df['cylinder'] = df['engine'].str.extract(r'(\d+\.?\d*) Cylinder').astype(float)              
            df['got_V'] = df['model'].str.extract(r'(\d+\.?\d*) V').notna().astype(int)
            return df
        settings.combined_df = encode_columns(settings.combined_df)
        settings.training_col_names += ['expensive_color_ext_encoded',
                                        'expensive_color_int_encoded']
        return settings

In [7]:
class FillNullValues:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings, numeric_fill=-1, category_fill='missing'):
        settings = deepcopy(original_settings)
        for col_name in settings.training_col_names:
            if pd.api.types.is_numeric_dtype(settings.combined_df[col_name]):
                settings.combined_df[col_name] = settings.combined_df[col_name].fillna(numeric_fill)
            else:
                settings.combined_df[col_name] = settings.combined_df[col_name].fillna(category_fill)
        return settings

In [8]:
class ConvertObjectToCategorical:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings):
        settings = deepcopy(original_settings)
        cat_cols = settings.categorical_col_names
        settings.combined_df[cat_cols] = settings.combined_df[cat_cols].astype('category')
        return settings

In [9]:
class LogTransformTarget:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings):
        settings = deepcopy(original_settings)
        target = settings.target_col_name
        settings.combined_df['log_' + target] = np.log(settings.combined_df[target] + 1)
        settings.target_col_name = 'log_' + target
        settings.logged = True
        return settings

In [10]:
train_csv_path = "data/used_car_prices/train_combined.csv"
test_csv_path = "data/used_car_prices/test.csv"
target_col_name = "price"

settings = DataSciencePipelineSettings(train_csv_path,
                                        test_csv_path,
                                        target_col_name)

In [16]:
settings.train_df[settings.target_col_name]

0           4200
1           4999
2          13900
3          45000
4          97500
           ...  
192537    349950
192538     53900
192539     90998
192540     62999
192541     40000
Name: price, Length: 192542, dtype: int64

In [20]:
round_thousand = (settings.train_df[settings.target_col_name]//1000)*1000
round_thousand, _ = round_thousand.factorize()
round_thousand

array([  0,   0,   1, ..., 163,   7,  66])

In [21]:
pd.Series(round_thousand, index=settings.train_df.index)

0           0
1           0
2           1
3           2
4           3
         ... 
192537     96
192538     44
192539    163
192540      7
192541     66
Length: 192542, dtype: int64

In [21]:
for col in settings.combined_df.columns:
    counts = settings.combined_df[col].value_counts().to_dict()
    # print(col, counts.values())
    print(np.quantile(list(counts.values()), 0.9))
    print(np.quantile(list(counts.values()), 0.95))

16788.199999999997
27597.799999999996
384.0
630.3499999999995
26103.5
27133.75
96.0
145.0
92125.99999999996
185953.99999999985
678.4000000000012
1223.8999999999965
18586.200000000004
31270.999999999996
345.1000000000001
895.3999999999955
559.6000000000001
4018.1999999999994
209019.2
226353.09999999998
255981.7
268979.85
253.0
472.1999999999989


In [54]:


full_transforms = [ConvertToLower.transform, 
                   BackPackerMinimalProcessing.transform, 
                   FillNullValues.transform, 
                   ConvertObjectToCategorical.transform]
# basic_transforms = [ConvertToLower.transform, FillNullValues.transform, ConvertObjectToCategorical.transform, LogTransformTarget.transform]

full_settings = reduce(lambda acc, func: func(acc), full_transforms, settings)
# basic_settings = reduce(lambda acc, func: func(acc), basic_transforms, settings)

In [183]:
# train_df, test_df = basic_settings.update()

# data_manager = KaggleDatasetManager(train_df,
#                                     basic_settings.training_col_names,
#                                     basic_settings.target_col_name,
#                                     0.9,
#                                     0.1,
#                                     0)

# (X_train, 
# X_valid, 
# X_test, 
# y_train, 
# y_valid,
# y_test) = data_manager.dataset_partition()


# # model = XGBRegressor(**{}, 
# #                     verbosity=0,
# #                     eval_metric='logloss',
# #                     tree_method='hist',
# #                     enable_categorical=True)

# model = ()

# num_splits = 10
# eval_metrics = {"r2" : r2_score, "rmse" : lambda y, yhat : root_mean_squared_error(np.exp(y), np.exp(yhat))}
# skf = KFold(n_splits=num_splits)

# model, cv_scores, test_scores = CrossValidateTestSklearnModel(model,
#                               eval_metrics,
#                               skf,
#                               num_splits).evaluate(X_train,
#                                                    y_train,
#                                                    X_test,
#                                                    y_test)

Final Model r2: 0.638607

10-fold cross validation r2:  0.6376903986890982

Final Model rmse: 70167.345097

10-fold cross validation rmse:  73703.32201325778

In [187]:
# from typing import Dict
# import numpy as np
# import pandas as pd
# from copy import deepcopy
# from sklearn.model_selection import StratifiedKFold
# from ktools.fitting.i_sklearn_model import ISklearnModel
# import lightgbm as lgb
# from lightgbm import log_evaluation, early_stopping


# class CrossValidateTestSklearnModel:

#     def __init__(self,
#                  sklearn_model_instance : ISklearnModel,
#                  evaluation_metrics : Dict[str, callable],
#                  kfold_object = None,
#                  num_splits : int = 5) -> None:
#         self.model = sklearn_model_instance
#         self._evaluation_metrics = evaluation_metrics
#         self._metric_names = list(evaluation_metrics.keys())
#         self._kf = kfold_object
#         self._num_metrics = len(self._metric_names)
#         self._num_splits = num_splits
#         self._model_list = []

#     def _fit_then_predict(self, X, y, X_test, y_test):
#         # model = deepcopy(self.model).fit(X, y)
        
#         train_data = lgb.Dataset(X, label=y)
#         val_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
#         model = lgb.train(model_params,
#                     train_data,
#                     valid_sets=[train_data, val_data],
#                     valid_names=['train', 'valid'],
#                     callbacks=callbacks    
#                     )
#         y_pred = model.predict(X_test)
#         return y_pred, model

#     def evaluate(self,
#                  X_train, y_train,
#                  X_test, y_test):

#         cv_results = np.zeros((self._num_splits, self._num_metrics))
#         cv_scores = None

#         if self._kf is not None:
#             for i, (train_index, val_index) in enumerate(self._kf.split(X_train, y_train)):
#                 X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
#                 y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

#                 y_pred, model = self._fit_then_predict(X_train_fold,
#                                                        y_train_fold,
#                                                        X_val_fold,
#                                                        y_val_fold)
#                 self._model_list += [model]

#                 for j, metric in enumerate(self._metric_names):
#                     score = self._evaluation_metrics[metric](np.array(y_val_fold), np.array(y_pred))
#                     cv_results[i][j] = score
            
#             cv_scores = pd.DataFrame(columns=self._metric_names, data=cv_results)
#             cv_scores.describe()

#         y_pred, self.model = self._fit_then_predict(X_train,
#                                                      y_train,
#                                                      X_test,
#                                                      y_test)
#         test_scores = {}
#         for j, metric in enumerate(self._metric_names):
#             score = self._evaluation_metrics[metric](np.array(y_test), np.array(y_pred))
#             test_scores[metric] = score
#             print(f"Final Model {metric}: {score:.6f}")
#             print(f"{self._num_splits}-fold cross validation {metric}: ", cv_results[:,j].mean())
        
#         return self._model_list, cv_scores, test_scores

In [55]:
train_df, test_df = full_settings.update()

In [78]:
# data_manager = KaggleDatasetManager(train_df,
#                                     full_settings.training_col_names,
#                                     full_settings.target_col_name,
#                                     0.9,
#                                     0.1,
#                                     0)

# (X_train, 
# X_valid, 
# X_test, 
# y_train, 
# y_valid,
# y_test) = data_manager.dataset_partition()


# model = XGBRegressor(**{}, 
#                     verbosity=0,
#                     eval_metric='rmse',
#                     tree_method='hist',
#                     enable_categorical=True)




# model = LGBMModel({    'learning_rate': 0.017521301504983752,
#     'max_depth': 42,
#     'reg_alpha': 0.06876635751774487, 
#     'reg_lambda': 9.738899198284985,
#     'num_leaves': 131,
#     'subsample': 0.2683765421728044,
#     'colsample_bytree': 0.44346036599709887,
#     'n_estimators': 1000,
#     'random_state': 42})

model = LGBMModel(
    
    **{'num_leaves': 426,
 'max_depth': 20,
 'learning_rate': 0.011353178352988012,
 'n_estimators': 884,
 'subsample': 0.5772552201954328,
 'colsample_bytree': 0.9164865430101521,
 'reg_alpha': 1.48699088003429e-06,
 'reg_lambda': 0.41539458543414265,
 'min_data_in_leaf': 73,
 'feature_fraction': 0.751673655170548,
 'bagging_fraction': 0.5120415391590843,
 'bagging_freq': 2,
 'min_child_weight': 0.017236362383443497,
 'cat_smooth': 54.81317407769262,
 'verbose' : -1}
 
 )

X = train_df.drop(columns="price")
y = train_df["price"]

# params = {'max_bin': 403, 
#         'learning_rate': 0.012720488589018275, 
#         'max_depth': 14, 
#         'num_boost_round': 921, 
#         'gamma': 7.327474792423768, 
#         'min_child_weight': 99.49960880266693, 
#         'subsample': 0.6815290497072164, 
#         'colsample_bytree': 0.6882587387019495, 
#         'colsample_bylevel': 0.6524817277480367, 
#         'colsample_bynode': 0.9692708790975624, 
#         'reg_alpha': 4.444851828081367e-06, 
#         'reg_lambda': 0.9647173450833559, 
#         'max_cat_threshold': 350, 
#         'grow_policy': 'lossguide'}

# params = {'max_bin': 431, 
#           'learning_rate': 0.010289485764739134, 
#           'max_depth': 21, 
#           'num_boost_round': 516, 
#           'gamma': 1.3067453666117153, 
#           'min_child_weight': 44.43942198348885, 
#           'subsample': 0.6167156245499883, 
#           'colsample_bytree': 0.6393032383785869, 
#           'colsample_bylevel': 0.5805205761409957, 
#           'colsample_bynode': 0.8741882369740219, 
#           'reg_alpha': 1.0166569230463967e-05, 
#           'reg_lambda': 0.46012423487591686, 
#           'max_cat_threshold': 122, 
#           'grow_policy': 'lossguide',
#           'booster' : 'dart'}

# params  ={'max_bin': 279, 
#           'learning_rate': 0.05411783467468672, 
#           'depth': 15, 
#           'iterations': 973, 
#           'bagging_temperature': 3.2498902422443727, 
#           'subsample': 0.7472867491636364, 
#           'colsample_bylevel': 0.9612415950869025, 
#           'min_data_in_leaf': 677.4758118043563, 
#           'l2_leaf_reg': 9.475872080718132, 
#           'grow_policy': 'Depthwise', 
#           'leaf_estimation_iterations': 1, 
#           'random_strength': 1.153902049647089, 
#           'leaf_estimation_method': 'Newton'}


# model = XGBoostModel(eval_verbosity=False,
#                      stopping_rounds=10,
#                      **params)

# model = CatBoostModel(stopping_rounds=10)

num_splits = 10
eval_metrics = {"r2" : r2_score, "rmse" : lambda y, yhat : root_mean_squared_error(y, yhat)}
skf = KFold(n_splits=num_splits, shuffle=True, random_state=42)

cvt = CrossValidateTestSklearnModel(model,
                              eval_metrics,
                              skf,
                              num_splits)

modellist, cv_scores, test_scores = cvt.evaluate(X,
                                                 y,
                                                 X,
                                                 y)

/Users/yuwei-1/anaconda3/envs/ktools_2/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 200 rounds
[150]	train's l2: 5.05793e+09	valid's l2: 6.90248e+09
[300]	train's l2: 4.83999e+09	valid's l2: 6.87395e+09
[450]	train's l2: 4.69867e+09	valid's l2: 6.86895e+09
[600]	train's l2: 4.57539e+09	valid's l2: 6.86855e+09
Early stopping, best iteration is:
[506]	train's l2: 4.65092e+09	valid's l2: 6.86412e+09


/Users/yuwei-1/anaconda3/envs/ktools_2/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 200 rounds
[150]	train's l2: 5.09179e+09	valid's l2: 4.05412e+09
[300]	train's l2: 4.87031e+09	valid's l2: 4.01722e+09
[450]	train's l2: 4.72347e+09	valid's l2: 4.0292e+09
Early stopping, best iteration is:
[294]	train's l2: 4.87767e+09	valid's l2: 4.01674e+09


/Users/yuwei-1/anaconda3/envs/ktools_2/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

Training until validation scores don't improve for 200 rounds
[150]	train's l2: 5.15185e+09	valid's l2: 5.60105e+09
[300]	train's l2: 4.93268e+09	valid's l2: 5.5506e+09
[450]	train's l2: 4.79175e+09	valid's l2: 5.54449e+09
Early stopping, best iteration is:
[388]	train's l2: 4.84842e+09	valid's l2: 5.54318e+09


/Users/yuwei-1/anaconda3/envs/ktools_2/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 200 rounds
[150]	train's l2: 5.01427e+09	valid's l2: 5.2856e+09
[300]	train's l2: 4.79999e+09	valid's l2: 5.24873e+09
[450]	train's l2: 4.65762e+09	valid's l2: 5.26385e+09
Early stopping, best iteration is:
[293]	train's l2: 4.80704e+09	valid's l2: 5.24728e+09


/Users/yuwei-1/anaconda3/envs/ktools_2/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 200 rounds
[150]	train's l2: 5.11284e+09	valid's l2: 5.60351e+09
[300]	train's l2: 4.89986e+09	valid's l2: 5.51753e+09
[450]	train's l2: 4.75628e+09	valid's l2: 5.51483e+09
Early stopping, best iteration is:
[362]	train's l2: 4.83679e+09	valid's l2: 5.50999e+09
Final Model r2: 0.215973
10-fold cross validation r2:  0.15267761774394395
Final Model rmse: 69788.662105
10-fold cross validation rmse:  72551.05395887098


In [80]:
params = {"max_bin": 403, 
                     "learning_rate": 0.012720488589018275, 
                     "max_depth": 14, 
                     "num_boost_round": 921, 
                     "gamma": 7.327474792423768, 
                     "min_child_weight": 99.49960880266693, 
                     "subsample": 0.6815290497072164, 
                     "colsample_bytree": 0.6882587387019495, 
                     "colsample_bylevel": 0.6524817277480367, 
                     "colsample_bynode": 0.9692708790975624, 
                     "reg_alpha": 4.444851828081367e-06, 
                     "reg_lambda": 0.9647173450833559, 
                     "max_cat_threshold": 350, 
                     "grow_policy": "lossguide",
                     "stopping_rounds" : 10}


model = XGBoostModel(eval_verbosity=False,
                     **params)

num_splits = 10
eval_metrics = {"r2" : r2_score, "rmse" : lambda y, yhat : root_mean_squared_error(y, yhat)}
skf = KFold(n_splits=num_splits, shuffle=True, random_state=42)

cvt = CrossValidateTestSklearnModel(model,
                              eval_metrics,
                              skf,
                              num_splits)

modellist, cv_scores, test_scores = cvt.evaluate(X,
                                                 y,
                                                 X,
                                                 y)

Final Model r2: 0.201357
10-fold cross validation r2:  0.1536242554137971
Final Model rmse: 70436.184051
10-fold cross validation rmse:  72510.51522461096


In [67]:
sub = pd.read_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/data/used_car_prices/sample_submission.csv', index_col=0)

for i, mdl in enumerate(modellist):
    sub[f'price_{i}'] = mdl.predict(test_df[full_settings.training_col_names])

sub['price'] = sub.drop(columns=['price']).mean(axis=1)
sub['price'].to_csv('submissions/used_cars/used_car_submission_v19.csv')

In [80]:
sub['price']

id
188533    18345.165615
188534    79120.161055
188535    54667.929227
188536    31480.856848
188537    29701.401953
              ...     
314218    30720.193571
314219    53239.669271
314220    20085.596915
314221    15467.033823
314222    42003.546320
Name: price, Length: 125690, dtype: float64

In [77]:
sub = pd.read_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/data/used_car_prices/sample_submission.csv', index_col=0)
sub1 = pd.read_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/submissions/used_cars/used_car_submission_v14.csv', index_col=0)
sub2 = pd.read_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/submissions/used_cars/used_car_submission_v18.csv', index_col=0)

In [90]:
(sub1*0.8+sub2*0.2).to_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/submissions/used_cars/used_car_submission_v24.csv')

In [89]:
(sub1*0.8+sub2*0.2)

,price
id,
188533,18237.890692
188534,79004.353844
188535,54261.162182
188536,31170.069879
188537,29735.015363
...,...
314218,30973.086857
314219,52579.858017
314220,20182.102932
